In [ ]:
!pip install torch
import torch
import OpenEXR, Imath

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/43/e5/2ddae60ae999b224aceb74490abeb885ee118227f866cb12046f0481d4c9/torch-2.3.0-cp310-cp310-manylinux1_x86_64.whl.metadata
  Using cached torch-2.3.0-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b43269810cd589ceaf5/typing_extensions-4.11.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/d2/05/e6600db80270777c4a64238a98d442f0fd07cc8915be2a1c16da7f2b9e74/sympy-1.12-py3-none-any.whl.metadata
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Obtaining dependency information for networkx from https://files.pythonhos

In [ ]:
# Add Generator Model path here

gen_model_path = "Models/best_gan_model.pt"

# Add Detector Model path here

det_model_path = "Models/best_detector_model.pt"

gen_model = torch.load(gen_model_path)
det_model = torch.load(det_model_path)

print(type(gen_model), "successfully loaded generator")
print(type(det_model), "successfully loaded detector")

In [ ]:
def load_exr(filename):
    """Load an EXR file and return as a NumPy array."""
    file = OpenEXR.InputFile(filename)
    dw = file.header()['dataWindow']
    size = (dw.max.x - dw.min.x + 1, dw.max.y - dw.min.y + 1)

    pt = Imath.PixelType(Imath.PixelType.FLOAT)
    channels = ['R', 'G', 'B']

    rgb = [np.frombuffer(file.channel(c, pt), dtype=np.float32) for c in channels]
    rgb = [np.reshape(c, (size[1], size[0])) for c in rgb]

    image = np.stack(rgb, axis=-1)
    # image = image.resize((1080,1920), Image.LANCZOS)
    return image

def hdr_normalize(img):
    hdr_max=65830.18848
    hdr_min=-326.18848
    real_A = F_normalize(img, 
                        [hdr_min, hdr_min, hdr_min],
                        [hdr_max, hdr_max, hdr_max])
    return real_A

In [ ]:
exr_image = load_exr(exr_path)
exr_image=torch.tensor(exr_image).permute(2,0,1)

png_image = Image.open(png_path)
png_image = np.array(png_image)

transform_img=transforms.Resize((1080,1920))

exr_image=transform_img(exr_image)